In [ ]:
!git clone https://github.com/adr143/dataset.git

Cloning into 'dataset'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 11.87 MiB | 14.12 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import joblib

In [ ]:
file_2022=pd.read_csv('REGION4A_2022.csv')
file_2023=pd.read_csv('REGION4A_2023.csv')
file_2024=pd.read_csv('REGION4A_2024.csv')

<ipython-input-3-ecd4255c9f85>:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  file_2023=pd.read_csv('REGION4A_2023.csv')


In [ ]:
col_needed = ['Date & Time', 'Rain - mm']

df_2022 = file_2022[col_needed]
df_2023 = file_2023[col_needed]
df_2024 = file_2024[col_needed]

In [ ]:
df_2023

,Date & Time,Rain - mm
0,01/01/2023 0:00,0.0
1,01/01/2023 0:15,0.0
2,01/01/2023 0:30,0.0
3,01/01/2023 0:45,0.0
4,01/01/2023 1:00,0.0
...,...,...
35035,12/31/2023 23:00,0.0
35036,12/31/2023 23:15,0.0
35037,12/31/2023 23:30,0.0
35038,12/31/2023 23:45,0.0


In [ ]:
combined_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

In [ ]:
combined_df['Date & Time'] = pd.to_datetime(combined_df['Date & Time'], format='%m/%d/%Y %H:%M')

In [ ]:
combined_df

,Date & Time,Rain - mm
0,2022-01-01 00:00:00,0.0
1,2022-01-01 00:15:00,0.0
2,2022-01-01 00:30:00,0.0
3,2022-01-01 00:45:00,0.0
4,2022-01-01 01:00:00,0.0
...,...,...
101235,2024-11-20 13:30:00,0.0
101236,2024-11-20 13:45:00,0.0
101237,2024-11-20 14:00:00,0.0
101238,2024-11-20 14:15:00,0.0


In [ ]:
combined_df.set_index('Date & Time', inplace=True)

combined_df['hour'] = combined_df.index.hour

for i in range(1, 4):
  combined_df[f'Rain - mm lag{i}'] = combined_df['Rain - mm'].shift(i)
combined_df = combined_df.dropna()

In [ ]:
combined_df

In [ ]:
for n in ['Rain - mm', 'Rain - mm lag1', 'Rain - mm lag2', 'Rain - mm lag3']:
  plt.plot(combined_df[n], alpha=0.2)

NameError: name 'plt' is not defined

In [ ]:
scaler_features = StandardScaler()
scaler_labels = StandardScaler()

feature_columns = ['hour', 'Rain - mm lag1', 'Rain - mm lag2', 'Rain - mm lag3']
scaler_features.fit(combined_df[feature_columns])
scaler_labels.fit(combined_df[['Rain - mm']])

def normalize_data(data):
    normalized_data = data.copy()

    # Apply transformations
    normalized_data[feature_columns] = scaler_features.transform(data[feature_columns])
    normalized_data[['Rain - mm']] = scaler_labels.transform(data[['Rain - mm']])

    return normalized_data

normalized_df = normalize_data(combined_df)

In [ ]:
inv_df = scaler_labels.inverse_transform(normalized_df[['Rain - mm']])

In [ ]:
joblib.dump(scaler_features, 'scaler_features.pkl')
joblib.dump(scaler_labels, 'scaler_labels.pkl')

['scaler_labels.pkl']

In [ ]:
inv_df

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
normalized_df

,Rain - mm,hour,Rain - mm lag1,Rain - mm lag2,Rain - mm lag3
Date & Time,,,,,
2022-01-01 00:45:00,-0.338848,-1.661041,-0.338848,-0.338848,-0.338848
2022-01-01 01:00:00,-0.338848,-1.516571,-0.338848,-0.338848,-0.338848
2022-01-01 01:15:00,-0.338848,-1.516571,-0.338848,-0.338848,-0.338848
2022-01-01 01:30:00,-0.338848,-1.516571,-0.338848,-0.338848,-0.338848
2022-01-01 01:45:00,-0.338848,-1.516571,-0.338848,-0.338848,-0.338848
...,...,...,...,...,...
2024-11-20 13:30:00,-0.338848,0.217070,-0.338848,-0.338848,-0.338848
2024-11-20 13:45:00,-0.338848,0.217070,-0.338848,-0.338848,-0.338848
2024-11-20 14:00:00,-0.338848,0.361540,-0.338848,-0.338848,-0.338848


In [ ]:
def create_lstm_sequences(data, sequence_length=4, target_column='Rain - mm'):
    sequences, labels = [], []

    for i in range(len(data) - sequence_length - 4):  # Ensure enough data for 4 future steps
        seq = data.iloc[i:i + sequence_length, :].values  # Past 4-time-step sequence

        # Get next 4 rainfall values as labels (scaled)
        label_scaled = data.iloc[i + sequence_length:i + sequence_length + 4][target_column].values

        sequences.append(seq)
        labels.append(label_scaled)  # Store 4 values instead of sum

    return np.array(sequences), np.array(labels)  # Labels are now shape (samples, 4)

In [ ]:
X, y = create_lstm_sequences(normalized_df, sequence_length=4, target_column='Rain - mm')

# Split dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((70862, 4, 5),
 (70862, 1),
 (20256, 4, 5),
 (20256, 1),
 (10114, 4, 5),
 (10114, 1))

In [ ]:
X_train[0]

array([[ 0.71965873,  1.0838898 ,  1.7781655 ,  1.5480553 ,  0.12137238],
       [ 1.1798791 ,  1.2283599 ,  0.71965873,  1.7781655 ,  1.5480553 ],
       [ 1.1798791 ,  1.2283599 ,  1.1798791 ,  0.71965873,  1.7781655 ],
       [-0.20078182,  1.2283599 ,  1.1798791 ,  1.1798791 ,  0.71965873]],
      dtype=float32)

In [ ]:
y_train[0]

array([ 1.17987905,  1.17987905, -0.20078182, -0.10873777])

In [ ]:
scaler_labels.inverse_transform(y_train.reshape(-1, 1))

array([[ 2.8],
       [10.4],
       [ 0. ],
       ...,
       [18.1],
       [ 0. ],
       [ 0. ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.regularizers import l2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization

model = Sequential()

# **🔹 CNN Feature Extraction**
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', input_shape=(4, 5)))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))  # Extra CNN Layer
model.add(Dropout(0.3))

# **🔹 LSTM for Temporal Learning**
model.add(LSTM(256, return_sequences=True))  # Increased LSTM units
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))  # Extra LSTM layer
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))  # Extra LSTM layer
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))  # Final LSTM layer
model.add(Dropout(0.2))

# **🔹 Fully Connected Layers**
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# **🔹 Output Layer (4 future predictions)**
model.add(Dense(4, activation='linear'))

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)                   │ (None, 4, 128)              │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_17 (Conv1D)                   │ (None, 4, 128)              │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_18 (Conv1D)                   │ (None, 4, 128)              │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_44 (Dropout)                 │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (None, 4, 256)              │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_45 (Dropout)                 │ (None, 4, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ (None, 4, 128)              │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_46 (Dropout)                 │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_25 (LSTM)                       │ (None, 4, 64)               │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_47 (Dropout)                 │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_26 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_48 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_49 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_50 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 767,105 (2.93 MB)

 Trainable params: 766,721 (2.92 MB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
cp = ModelCheckpoint('model1/best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.000001),
    metrics=[RootMeanSquaredError(), 'mae']
)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=520, callbacks=[early_stopping, reduce_lr, cp], batch_size=1024)

Epoch 1/520
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - loss: 0.8819 - mae: 0.5305 - root_mean_squared_error: 0.9390 - val_loss: 1.1989 - val_mae: 0.5610 - val_root_mean_squared_error: 1.0949 - learning_rate: 1.0000e-06
Epoch 2/520
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.7962 - mae: 0.4706 - root_mean_squared_error: 0.8919 - val_loss: 1.1968 - val_mae: 0.5527 - val_root_mean_squared_error: 1.0940 - learning_rate: 1.0000e-06
Epoch 3/520
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.6486 - mae: 0.4190 - root_mean_squared_error: 0.8052 - val_loss: 1.1920 - val_mae: 0.5420 - val_root_mean_squared_error: 1.0918 - learning_rate: 1.0000e-06
Epoch 4/520
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.5675 - mae: 0.3794 - root_mean_squared_error: 0.7532 - val_loss: 1.1818 - val_mae: 0.5268 - val_root_mean_squared_error: 1.0871 - learning_rate: 1.0000e-06
Epoch 5/520
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.5607 - mae: 0.3615 - root_mean_squared_error: 0.7486 - val_loss: 1

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model('best_model.keras')

In [ ]:
model1.input_shape

(None, 4, 5)

In [ ]:
# Make predictions on validation set
val_predictions = model1.predict(X_val)  # Shape: (num_samples, 4)

# Compute metrics for each step (4 intervals)
val_rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
val_mae = mean_absolute_error(y_val, val_predictions)
val_r2 = r2_score(y_val, val_predictions)

# Print metrics
print(f"Validation Metrics for 15-min interval predictions:")
for i in range(4):
    rmse_i = np.sqrt(mean_squared_error(y_val[:, i], val_predictions[:, i]))
    mae_i = mean_absolute_error(y_val[:, i], val_predictions[:, i])
    r2_i = r2_score(y_val[:, i], val_predictions[:, i])

    print(f"  Interval {i+1} (t+{(i+1)*15} min): RMSE={rmse_i:.4f}, MAE={mae_i:.4f}, R²={r2_i:.4f}")

# Overall performance across all 4 intervals
print(f"\nOverall Validation Metrics:")
print(f"  RMSE: {val_rmse:.4f}")
print(f"  MAE: {val_mae:.4f}")
print(f"  R²: {val_r2:.4f}")


In [ ]:
# Make predictions on test set
test_predictions = model.predict(X_test)  # Shape: (num_samples, 4)

# Compute metrics for each step (4 intervals)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print metrics for each interval
print(f"Test Metrics for 15-min interval predictions:")
for i in range(4):
    rmse_i = np.sqrt(mean_squared_error(y_test[:, i], test_predictions[:, i]))
    mae_i = mean_absolute_error(y_test[:, i], test_predictions[:, i])
    r2_i = r2_score(y_test[:, i], test_predictions[:, i])

    print(f"  Interval {i+1} (t+{(i+1)*15} min): RMSE={rmse_i:.4f}, MAE={mae_i:.4f}, R²={r2_i:.4f}")

# Overall performance across all 4 intervals
print(f"\nOverall Test Metrics:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  R²: {test_r2:.4f}")

In [ ]:
scaler_features = joblib.load('scaler_features.pkl')
scaler_labels = joblib.load('scaler_labels.pkl')

In [ ]:
print(scaler_labels.mean_)  # Check the mean of the scaler for the target
print(scaler_labels.scale_)  # Check the scaling factor

[2.94532594]
[8.20168479]


In [ ]:
print("Shape of y_test:", y_test.shape)
print("Shape of test_predictions:", test_predictions.shape)

Shape of y_test: (10113,)
Shape of test_predictions: (10113,)


In [ ]:
y_test

array([ 1.81117349, -0.35911231, -0.35911231, ..., -0.35911231,
       -0.35911231,  1.81117349])

In [ ]:
# Reshape predictions to match scaler expectations
y_test_reshaped = y_test.reshape(-1, 4)  # (samples, 4)
y_predicted = test_predictions.reshape(-1, 4)  # (samples, 4)

# Inverse transform predictions & labels
y_test_original = scaler_labels.inverse_transform(y_test_reshaped)
test_predictions_original = scaler_labels.inverse_transform(y_predicted)

# Ensure no negative rainfall values
test_predictions_original = np.maximum(test_predictions_original, 0)

# Print Actual vs Predicted Values (for first few samples)
print("\nActual vs Predicted Values (Rainfall in mm):")
for i in range(len(y_test_original)):  # Iterate over all test samples
    print(f"Sample {i+1}:")
    for j in range(4):  # 4 intervals (15, 30, 45, 60 min)
        print(f"  t+{(j+1)*15} min | Actual: {y_test_original[i, j]:.2f} mm, Predicted: {test_predictions_original[i, j]:.2f} mm")
    print("-" * 40)  # Separator for clarity



Streaming output truncated to the last 5000 lines.
  t+60 min | Actual: 0.00 mm, Predicted: 0.05 mm
----------------------------------------
Sample 9281:
  t+15 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+30 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+45 min | Actual: 0.00 mm, Predicted: 0.03 mm
  t+60 min | Actual: 0.00 mm, Predicted: 0.06 mm
----------------------------------------
Sample 9282:
  t+15 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+30 min | Actual: 0.00 mm, Predicted: 0.01 mm
  t+45 min | Actual: 0.00 mm, Predicted: 0.03 mm
  t+60 min | Actual: 0.00 mm, Predicted: 0.07 mm
----------------------------------------
Sample 9283:
  t+15 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+30 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+45 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+60 min | Actual: 0.00 mm, Predicted: 0.00 mm
----------------------------------------
Sample 9284:
  t+15 min | Actual: 0.00 mm, Predicted: 0.00 mm
  t+30 min | Actual: 0.00 mm, Predicted: 0.00 m

In [ ]:
model1.evaluate(X_test, y_test)

# Closeness to Actual, Margin of Error in mm

317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3746 - mae: 0.1837 - root_mean_squared_error: 0.6100


[0.40732529759407043, 0.6382203698158264, 0.18265357613563538]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Inverse transform the scaled predictions and actual values
y_test_original = scaler_labels.inverse_transform(y_test.reshape(-1, 1)).flatten()
test_predictions_original = scaler_labels.inverse_transform(test_predictions.reshape(-1, 1)).flatten()

# Plot Actual vs Predicted Rainfall in mm
plt.figure(figsize=(10, 6))
plt.plot(y_test_original, label='Actual', color='blue', marker='o', linestyle='None', alpha=0.3)
plt.plot(test_predictions_original, label='Predicted', color='red', marker='x', linestyle='None', alpha=0.3)
plt.title('1-Hour - Actual vs Predicted Rainfall (mm)')
plt.xlabel('Data Point')
plt.ylabel('Rainfall (mm)')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Save the model
import os
model_file = "rainfall_mm_v8_with_cnn"
model_1hr_filename = f'models/{model_file}.h5'
model.save(model_1hr_filename)
print(f"15-Minute Prediction Model saved as '{os.path.abspath(model_1hr_filename)}'")

15-Minute Prediction Model saved as '/content/models/rainfall_mm_v8_with_cnn.h5'


In [ ]:
with open(f"models/{model_file}.txt", "w") as file:
    file.write(f"1-Hour Validation Metrics:\n")
    file.write(f"  RMSE: {val_rmse:.4f}\n")
    file.write(f"  MAE: {val_mae:.4f}\n")
    file.write(f"  R²: {val_r2:.4f}\n\n")

    file.write(f"1-Hour Test Metrics:\n")
    file.write(f"  RMSE: {test_rmse:.4f}\n")
    file.write(f"  MAE: {test_mae:.4f}\n")
    file.write(f"  R²: {test_r2:.4f}\n")

In [ ]:
!zip -r model.zip /content/model1

updating: content/model1/ (stored 0%)
  adding: content/model1/best.keras (deflated 7%)
  adding: content/model1/best_model.keras (deflated 7%)
  adding: content/model1/something_model.h5 (deflated 7%)
